#### Learning SQLAlchemy

https://www.sqlalchemy.org/library.html#tutorials

In [3]:
import sqlalchemy
print(sqlalchemy.__version__)

1.1.9


![SQLAlchemy Structure Overview](https://image.slidesharecdn.com/sqlaintro-130921142257-phpapp02/95/michael-bayer-introduction-to-sqlalchemy-postgres-open-6-638.jpg)

In [34]:
import sqlite3
conn = sqlite3.connect('memory.db')
c = conn.cursor()

def execute_sql_script(filename):
    with open(filename, 'r') as fd:
        sqlFile = fd.read()

    sqlCommands = sqlFile.split(';')

    for command in sqlCommands:
        try:
            c.execute(command)
        except sqlite3.OperationalError as msg:
            print("Command skipped: ", msg)

In [35]:
execute_sql_script('cars.sql')
execute_sql_script('author_books.sql')

Command skipped:  cannot commit - no transaction is active
Command skipped:  cannot commit - no transaction is active


In [36]:
# List tables in databse
c.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(c.fetchall())
c.close()

[('Cars',), ('Authors',), ('Books',)]


In [43]:
from sqlalchemy import create_engine
eng = create_engine('sqlite:///memory')
with eng.connect() as conn:
    rs = conn.execute('SELECT 5')
    data = rs.fetchone()[0]
    print("Data: %s" % data)

Data: 5


In [44]:
from sqlalchemy.sql import text
with eng.connect() as conn:
    conn.execute(text('DROP TABLE IF EXISTS Cars'))
    conn.execute(text('''CREATE TABLE Cars(Id INTEGER PRIMARY KEY,
    Name TEXT, Price INTEGER)'''))
    
    data = ( { "Id": 1, "Name": "Audi", "Price": 52642 },
             { "Id": 2, "Name": "Mercedes", "Price": 57127 },
             { "Id": 3, "Name": "Skoda", "Price": 9000 },
             { "Id": 4, "Name": "Volvo", "Price": 29000 },
             { "Id": 5, "Name": "Bentley", "Price": 350000 },
             { "Id": 6, "Name": "Citroen", "Price": 21000 },
             { "Id": 7, "Name": "Hummer", "Price": 41400 },
             { "Id": 8, "Name": "Volkswagen", "Price": 21600 }
    )
    
    for line in data:
        conn.execute(text('''INSERT INTO Cars(Id, Name, Price) VALUES (:Id, :Name, :Price)'''), **line)

In [105]:
from prettytable import PrettyTable, from_db_cursor
def print_table(eng, table_name):
    conn = eng.raw_connection()
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM %s' % table_name)
    y = from_db_cursor(cursor)
    y.padding_width = 1
    y.align[y.field_names[1]]="l"
    y.align[y.field_names[-1]]="r"
    print(y)
    conn.close()

In [106]:
print_table(eng, 'Cars')

+----+------------+--------+
| Id | Name       |  Price |
+----+------------+--------+
| 1  | Audi       |  52642 |
| 2  | Mercedes   |  57127 |
| 3  | Skoda      |   9000 |
| 4  | Volvo      |  29000 |
| 5  | Bentley    | 350000 |
| 6  | Citroen    |  21000 |
| 7  | Hummer     |  41400 |
| 8  | Volkswagen |  21600 |
+----+------------+--------+


In [82]:
from sqlalchemy import create_engine, Table, Column, Integer, String, MetaData

meta = MetaData()
cars = Table('Cars', meta, 
             Column('Id', Integer, primary_key=True),
             Column('Name', String),
             Column('Price', Integer)
            )

In [79]:
print("Columns:")
for col in cars.c:
    print(col)
    
print("\nPrimary Keys:")
for pk in cars.primary_key:
    print(pk)

Columns:
Cars.Id
Cars.Name
Cars.Price

Primary Keys:
Cars.Id


In [77]:
col_id = cars.c.Id
col_id.name, col_id.type, col_id.nullable, col_id.primary_key

('Id', Integer(), False, True)

In [84]:
# reflect() - Sync Metadata object with database
meta = MetaData()
meta.reflect(bind=eng)

for table in meta.tables:
    print(table)

Authors
Cars
Books


In [111]:
# inspect() - Inspector: Low-level database schema inspection
from sqlalchemy import inspect

insp = inspect(eng)
insp.get_table_names(), insp.get_columns("Cars"), insp.get_primary_keys("Cars"), insp.get_schema_names()

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: SADeprecationWarning: Call to deprecated method get_primary_keys.  Use get_pk_constraint instead.


(['Authors', 'Books', 'Cars'],
 [{'autoincrement': 'auto',
   'default': None,
   'name': 'Id',
   'nullable': True,
   'primary_key': 1,
   'type': INTEGER()},
  {'autoincrement': 'auto',
   'default': None,
   'name': 'Name',
   'nullable': True,
   'primary_key': 0,
   'type': TEXT()},
  {'autoincrement': 'auto',
   'default': None,
   'name': 'Price',
   'nullable': True,
   'primary_key': 0,
   'type': INTEGER()}],
 ['Id'],
 ['main'])

In [133]:
# select() == SELECT
from sqlalchemy.sql import select
from tabulate import tabulate

with eng.connect() as conn:
    meta = MetaData(eng)
    cars = Table('Cars', meta, autoload=True)
    
    stmt = select([cars])
    rs = conn.execute(stmt)
    print(tabulate(rs.fetchall()))
    # print(rs.fetchall())

-  ----------  ------
1  Audi         52642
2  Mercedes     57127
3  Skoda         9000
4  Volvo        29000
5  Bentley     350000
6  Citroen      21000
7  Hummer       41400
8  Volkswagen   21600
-  ----------  ------


In [128]:
# select() - Selecting specific columns and limiting results
with eng.connect() as conn:
    meta = MetaData(eng)
    cars = Table('Cars', meta, autoload=True)
    
    stmt = select([cars.c.Name, cars.c.Price]).limit(3)
    rs = conn.execute(stmt)
    
    print(tabulate(rs.fetchall()))

--------  -----
Audi      52642
Mercedes  57127
Skoda      9000
--------  -----


In [130]:
# select() - Using where clause
from sqlalchemy.sql import and_

with eng.connect() as conn:
    meta = MetaData(eng)
    cars = Table('Cars', meta, autoload=True)
    
    stmt = select([cars]).where(and_(cars.c.Price > 10000, cars.c.Price < 40000))
    rs = stmt.execute()
    
    print(tabulate(rs.fetchall()))

-  ----------  -----
4  Volvo       29000
6  Citroen     21000
8  Volkswagen  21600
-  ----------  -----


In [134]:
# select() - Filtering output using where and 'like'
with eng.connect() as conn:
    meta = MetaData(eng)
    cars = Table('Cars', meta, autoload=True)
    
    stmt = select([cars]).where(cars.c.Name.like('%en'))
    rs = conn.execute(stmt)
    
    print(tabulate(rs.fetchall()))

-  ----------  -----
6  Citroen     21000
8  Volkswagen  21600
-  ----------  -----


In [138]:
# order_by() == ORDER BY clause
from sqlalchemy.sql import asc

with eng.connect() as conn:
    meta = MetaData(eng)
    cars = Table('Cars', meta, autoload=True)
    
    stmt = select([cars]).order_by(asc(cars.c.Name))
    rs = conn.execute(stmt)
    
    print(tabulate(rs.fetchall()))

-  ----------  ------
1  Audi         52642
5  Bentley     350000
6  Citroen      21000
7  Hummer       41400
2  Mercedes     57127
3  Skoda         9000
8  Volkswagen   21600
4  Volvo        29000
-  ----------  ------


In [139]:
# in_() == IN clause

from sqlalchemy import tuple_

with eng.connect() as conn:
    meta = MetaData(eng)
    cars = Table('Cars', meta, autoload=True)
    
    k = [(2,), (4,), (6,), (8,)]
    stmt = select([cars]).where(tuple_(cars.c.Id).in_(k))
    rs = conn.execute(stmt)
    
    print(tabulate(rs.fetchall()))

-  ----------  -----
2  Mercedes    57127
4  Volvo       29000
6  Citroen     21000
8  Volkswagen  21600
-  ----------  -----


In [148]:
# Inserting values using insert()
with eng.connect() as conn:
    meta = MetaData(eng)
    
    drop_stmt = 'DROP TABLE IF EXISTS %s' % 'Cars'
    _ = conn.execute(drop_stmt)
    
    cars = Table('Cars', meta,
                 Column('Id', Integer, primary_key=True),
                 Column('Name', String),
                 Column('Price', Integer)
                )
    cars.create()
    
    ins1 = cars.insert().values(Id=1, Name='Audi', Price=52642)
    conn.execute(ins1)
    ins2 = cars.insert().values(Id=2, Name='Mercedes', Price=57127)
    conn.execute(ins2)
    ins3 = cars.insert().values(Id=3, Name='Skoda', Price=6000)
    conn.execute(ins3)
    
    stmt = select([cars])
    rs = conn.execute(stmt)
    
    print(tabulate(rs.fetchall()))

-  --------  -----
1  Audi      52642
2  Mercedes  57127
3  Skoda      6000
-  --------  -----


In [152]:
# join()
from sqlalchemy import ForeignKey

with eng.connect() as conn:
    meta = MetaData(eng)
    
    authors = Table('Authors', meta, autoload=True)
    books = Table('Books', meta, autoload=True)
    
    stmt = select([authors.join(books)])
    rs = conn.execute(stmt)
    
    print(tabulate(rs.fetchall()))

-  ---------------  -  -----------------  -
1  Jane Austen      1  Emma               1
2  Leo Tolstoy      2  War and Peace      2
3  Joseph Heller    3  Catch XII          3
4  Charles Dickens  4  David Copperfield  4
3  Joseph Heller    5  Good as Gold       3
2  Leo Tolstoy      6  Anna Karenia       2
-  ---------------  -  -----------------  -


SQLAlchemy ORM (Object Relational Mapper)  
- Classes to Tables
- Instance Objects to Rows
- Instance Attributes to Columns

In [172]:
# ORM in SQLAlchemy - Example using a Car class

from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

Base = declarative_base()

conn = eng.connect()
drop_stmt = 'DROP TABLE IF EXISTS %s' % 'Cars'
_ = conn.execute(drop_stmt)
conn.close()

class Car(Base):
    __tablename__ = 'Cars'
    
    Id = Column(Integer, primary_key=True)
    Name = Column(String)
    Price = Column(Integer)
    
Base.metadata.bind = eng
Base.metadata.create_all()

Session = sessionmaker(bind=eng)
ses = Session()

ses.add_all(
[
    Car(Id=1, Name='Audi', Price=52642),
    Car(Id=2, Name='Mercedes', Price=57127),
    Car(Id=3, Name='Skoda', Price=9000),
    Car(Id=4, Name='Volvo', Price=29000),
    Car(Id=5, Name='Bentley', Price=250000),
    Car(Id=6, Name='Citroen', Price=21000),
    Car(Id=7, Name='Hummer', Price=41400),
    Car(Id=8, Name='Volkswagen', Price=21600),
])
ses.commit()

rs = ses.query(Car).all()

for car in rs:
    print(car.Name, car.Price)

Audi 52642
Mercedes 57127
Skoda 9000
Volvo 29000
Bentley 250000
Citroen 21000
Hummer 41400
Volkswagen 21600


In [173]:
# Adding a new car to the table

c1 = Car(Name='Oldsmobile', Price=23450)
ses.add(c1)
ses.commit()

rs = ses.query(Car).all()

for car in rs:
    print(car.Name, car.Price)

Audi 52642
Mercedes 57127
Skoda 9000
Volvo 29000
Bentley 250000
Citroen 21000
Hummer 41400
Volkswagen 21600
Oldsmobile 23450


In [174]:
# filter()
rs = ses.query(Car).filter(Car.Name.like('%en'))
for car in rs:
    print(car.Name, car.Price)
print('\n', rs)

Citroen 21000
Volkswagen 21600

 SELECT "Cars"."Id" AS "Cars_Id", "Cars"."Name" AS "Cars_Name", "Cars"."Price" AS "Cars_Price" 
FROM "Cars" 
WHERE "Cars"."Name" LIKE ?


In [175]:
rs = ses.query(Car).filter(Car.Id.in_([2,4,6,8]))
for car in rs:
    print(car.Id, car.Name, car.Price)

2 Mercedes 57127
4 Volvo 29000
6 Citroen 21000
8 Volkswagen 21600


In [192]:
from sqlalchemy.orm import relationship

Base = declarative_base()

class Author(Base):
    __tablename__ = "Authors" 
    
    AuthorId = Column(Integer, primary_key=True)
    Name = Column(String)
    Books = relationship("Book")
    
class Book(Base):
    __tablename__ = "Books"
    
    BookId = Column(Integer, primary_key=True)
    Title = Column(String)
    AuthorId = Column(Integer, ForeignKey("Authors.AuthorId"))
    
    Author = relationship("Author")
    
res = ses.query(Author).filter(Author.Name=="Leo Tolstoy").first()

for book in res.Books:
    print(book.Title)

print()
res = ses.query(Book).filter(Book.Title=="Emma").first()
print(res.Author.Name)

War and Peace
Anna Karenia

Jane Austen
